In [1]:
pip install --upgrade numexpr

     ---------------------------------------- 95.0/95.0 kB 5.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install boto3


     -------------------------------------- 139.3/139.3 kB 4.2 MB/s eta 0:00:00
     --------------------------------------- 11.8/11.8 MB 36.4 MB/s eta 0:00:00
     ---------------------------------------- 82.0/82.0 kB ? eta 0:00:00
     ---------------------------------------- 143.8/143.8 kB ? eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.5
    Uninstalling urllib3-2.0.5:
      Successfully uninstalled urllib3-2.0.5
Note: you may need to restart the kernel to use updated packages.


In [6]:
# requirements
import os
import boto3
import sagemaker
import numpy as np
import pandas as pd

In [7]:
# read rating data
music_path = './dataset/Music Info.csv'
music_info = pd.read_csv(music_path)

listening_path = './dataset/User Listening History.csv'
listening_info = pd.read_csv(listening_path)

#print(music_info.head())
#print(listening_info.head())

# cutting dataset 
music_info = music_info.head(50000)
listening_info =listening_info.head(50000)

In [8]:
# Merge the data. 
left = music_info.set_index('track_id')
right = listening_info.set_index('track_id')
music_listening_info = left.join(right)

music_listening_info.drop(columns=['genre', 'tags'])
music_listening_info.dropna(inplace=True) # 결측치 제거

In [9]:
music_listening_info.head(10)

,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,user_id,playcount
track_id,,,,,,,,,,,,,,,,,,,,,
TRAAAED128E0783FAB,It's About Time,Jamie Cullum,https://p.scdn.co/mp3-preview/9bb7b58f848a742b...,00ZCW9lUa2bqwpF6rhtaMI,"jazz, piano, chill",Jazz,2004,247000,0.565,0.507,...,1,0.0541,0.702000,0.000066,0.1290,0.454,152.462,4,cf7bd4b5b398b3e150cf262d79147312a69b96ac,9.0
TRAAAED128E0783FAB,It's About Time,Jamie Cullum,https://p.scdn.co/mp3-preview/9bb7b58f848a742b...,00ZCW9lUa2bqwpF6rhtaMI,"jazz, piano, chill",Jazz,2004,247000,0.565,0.507,...,1,0.0541,0.702000,0.000066,0.1290,0.454,152.462,4,043d81932e75d5749ed5758d6420506e7bc457a5,3.0
TRAABJS128F9325C99,Auburn and Ivory,Beach House,https://p.scdn.co/mp3-preview/b3583a8703729f6a...,0r8EsO3ylF81gjbFFVlSwX,"indie, downtempo",Rock,2006,270026,0.438,0.285,...,0,0.0265,0.866000,0.000003,0.0696,0.366,145.601,3,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,3.0
TRAABJS128F9325C99,Auburn and Ivory,Beach House,https://p.scdn.co/mp3-preview/b3583a8703729f6a...,0r8EsO3ylF81gjbFFVlSwX,"indie, downtempo",Rock,2006,270026,0.438,0.285,...,0,0.0265,0.866000,0.000003,0.0696,0.366,145.601,3,cedc33e1560c8520901d42c9fac45fada19a73ef,1.0
TRAABJS128F9325C99,Auburn and Ivory,Beach House,https://p.scdn.co/mp3-preview/b3583a8703729f6a...,0r8EsO3ylF81gjbFFVlSwX,"indie, downtempo",Rock,2006,270026,0.438,0.285,...,0,0.0265,0.866000,0.000003,0.0696,0.366,145.601,3,e12ff6d809877f9158135ef7c3b48eb28c5174bd,3.0
TRAABJS128F9325C99,Auburn and Ivory,Beach House,https://p.scdn.co/mp3-preview/b3583a8703729f6a...,0r8EsO3ylF81gjbFFVlSwX,"indie, downtempo",Rock,2006,270026,0.438,0.285,...,0,0.0265,0.866000,0.000003,0.0696,0.366,145.601,3,ea4996c283bc5306c0c55181815107218cd21df3,1.0
TRAABJS128F9325C99,Auburn and Ivory,Beach House,https://p.scdn.co/mp3-preview/b3583a8703729f6a...,0r8EsO3ylF81gjbFFVlSwX,"indie, downtempo",Rock,2006,270026,0.438,0.285,...,0,0.0265,0.866000,0.000003,0.0696,0.366,145.601,3,556f497c582bef15f32be8876560b3cda2df0837,2.0
TRAABJS128F9325C99,Auburn and Ivory,Beach House,https://p.scdn.co/mp3-preview/b3583a8703729f6a...,0r8EsO3ylF81gjbFFVlSwX,"indie, downtempo",Rock,2006,270026,0.438,0.285,...,0,0.0265,0.866000,0.000003,0.0696,0.366,145.601,3,bd0f6d41bef28fe65e68b18da9361c316ba0a8a7,1.0
TRAABJS128F9325C99,Auburn and Ivory,Beach House,https://p.scdn.co/mp3-preview/b3583a8703729f6a...,0r8EsO3ylF81gjbFFVlSwX,"indie, downtempo",Rock,2006,270026,0.438,0.285,...,0,0.0265,0.866000,0.000003,0.0696,0.366,145.601,3,002543003041db1d049206b09426d5cdffc0f451,1.0


### preprocessing
##### scaling numerical features 

In [10]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# numerical_features = music_listening_info.select_dtypes(include=['float', 'int']).columns.tolist()
numerical_features = ['danceability', 'energy', 'key', 'loudness','speechiness', 'acousticness', 'instrumentalness', 'liveness',
'valence', 'tempo']
scaler = MinMaxScaler()
music_listening_info[numerical_features] = scaler.fit_transform(music_listening_info[numerical_features])

feature_cols = ['danceability', 'energy', 'key', 'loudness',
      'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'artist']


# Content based filtering 

In [11]:
uni_song = music_listening_info['name'].unique()
uni_song

array(["It's About Time", 'Auburn and Ivory', 'Double Feature', ...,
       'Solo Dolo (Nightmare)', 'Flutter Girl', 'Lord Anthony'],
      dtype=object)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

# Create a per-song feature vector
song_vectors = {}
song_set = music_listening_info['name'].unique()
for song in song_set:
    song_df = music_listening_info[music_listening_info['name'] == song].reset_index(drop=False)

    feature_cols = ['danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature'] # feature_cols can represent song's attributes 
    song_features = song_df[feature_cols]
    song_vectors[song] = list(song_features.mean().values.astype(float))

#  calculate song's similarity 
song_similarity = cosine_similarity(list(song_vectors.values()))

# 
"""
def recommend_song (song_name, n ) :
This function is passed a song title and n, where n is the number of songs to be recommended. 
As a result, it recommends n similar songs. 

input parameter : song name (String) , n (input) 
return values : similar_songs[:n]

"""
def recommend_song(song_name, n):
    if song_name not in song_vectors:
        return "Song not found"

    song_index = list(song_set).index(song_name)
    similar_indices = song_similarity[song_index].argsort()[-n-1:][::-1]
    similar_songs = [song_set[i] for i in similar_indices if song_set[i] != song_name]

    return similar_songs[:n]


#print result: 
print(recommend_song('One Last Time', 10))

['Twentysomething', "You'll Always Find Me In The Kitchen At Parties", 'I Do (Cherish You)', 'Cross Bones Style', 'Underneath Your Clothes', 'This Bitter Pill', 'There Is So Much More', "I'm Gonna Miss Her", "Just Like Tom Thumb's Blues", "Ain't No Reason"]


# Collaborative filtering 

In [13]:
user_uni = music_listening_info['user_id'].unique()
user_uni

array(['cf7bd4b5b398b3e150cf262d79147312a69b96ac',
       '043d81932e75d5749ed5758d6420506e7bc457a5',
       '01655ae6bc52e29c9cd100a7dde4e9eeae5e4031', ...,
       'a14c04b4fa96b7845bf1b3bff8881c27ca0c8c93',
       '87258fa894f0bbd1636c77ec10a0646e3f731892',
       'dd9fc4c46af43696cb8bdda1de3c6e4908f3872a'], dtype=object)

In [14]:
import pandas as pd
from scipy.sparse import csr_matrix

# Load data
# data = pd.read_csv('your_data.csv') # Path to your data file

# Create a user-item rating matrix
user_item_matrix = music_listening_info.pivot_table(index='user_id', columns='spotify_id', values='playcount', fill_value=0)

# Create a mapping of track IDs to song titles
track_title_mapping = music_listening_info[['spotify_id', 'name']].drop_duplicates().set_index('spotify_id')['name'].to_dict()

# Convert the matrix to a sparse matrix format
sparse_user_item = csr_matrix(user_item_matrix)

# Calculate similarity
user_similarity = cosine_similarity(sparse_user_item)

def recommend_songs_for_user(user_id, n_recommendations=5):
    """
    This function recommends songs for a given user based on user similarity. It first checks if the user exists in the database. 
    If the user exists, it finds similar users and recommends songs liked by these similar users but not yet rated by the target user. 
    The function returns the top N recommended song titles.
    
    Parameters:
    user_id (str): The ID of the user for whom the recommendation is to be made.
    n_recommendations (int): The number of song recommendations to return. Default is 5.

    Returns:
    list: A list of recommended song titles.
    """
    if user_id not in user_item_matrix.index:
        return "User not found"

    user_index = user_item_matrix.index.get_loc(user_id)
    similar_users = user_similarity[user_index].argsort()[::-1][1:]

    # Songs currently rated by the user
    user_items_rated = set(user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index)

    # Recommend items liked by similar users
    recommended_items = {}
    for similar_user_index in similar_users:
        similar_user_id = user_item_matrix.index[similar_user_index]
        similar_user_items = user_item_matrix.loc[similar_user_id]

        # Consider only items not yet rated by the current user
        for item in set(similar_user_items[similar_user_items > 0].index) - user_items_rated:
            recommended_items[item] = recommended_items.get(item, 0) + similar_user_items[item]

    # Recommend items with the highest ratings
    recommended_items_sorted = sorted(recommended_items.items(), key=lambda x: x[1], reverse=True)

    # Convert recommended song IDs to titles
    recommended_song_titles = [track_title_mapping[item[0]] for item in recommended_items_sorted[:n_recommendations]]
    return recommended_song_titles

# Recommendations for a specific user
user_id = 'cf7bd4b5b398b3e150cf262d79147312a69b96ac'

recommendations = recommend_songs_for_user(user_id, 10)
print(recommendations)



['Revelry', 'Alejandro', 'The Good Fight', 'Bring Me To Life', 'Heartbreak Warfare', 'Float On', "Reelin' In The Years", 'Party In The U.S.A.', 'Hey There Delilah', 'The Past Should Stay Dead']
